# Reporte Solicitudes Ara

In [ ]:
from_drive=True

In [ ]:
## GIT ON COLAB ONLY
try:
    from google.colab import userdata
    git_token = userdata.get('gitToken')
    git_user = userdata.get('gitUser')
    git_url = f'https://{git_token}@github.com/rene-aum/Atlas.git'
    branch_to_pull = 'dev'
    !git clone {git_url}
    %cd Atlas
    !git checkout -b colab_branch
    !git pull origin {branch_to_pull}
    !pip install -r PipelinesConsumo/src/requirements.txt
    %cd PipelinesConsumo
except Exception as e:
    print(e)
    print('Running in other environment not colab probably!')

In [ ]:
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pytz
# from matplotlib.ticker import FuncFormatter
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.utils import (get_dates_dataframe,
                       add_year_week,
                       custom_read,
                       process_columns)
from PipelinesConsumo.src.rawAtlas import RawAtlas
from PipelinesConsumo.src.processedAtlas import ProcessedAtlas
from src.transformed import Transformed
from utils.drive_toolbox import(from_drive_to_local,
                             get_last_modification_date_drive,
                             create_sheets_in_drive_folder,
                             update_sheets_in_drive_folder,
                             read_from_google_sheets,
                             list_file_ids_for_drive_folder,
                             create_csv_file_in_drive_folder,
                             write_csv_to_drive,
                             read_csv_from_drive)
from src.constants import (atlas_raw_output_folder_id,
                           atlas_consumo_output_folder_id,
                           consumo_sheets_ids_dict,
                           data_source_folder_id,
                           raw_output_ids,
                           folder_id_bauto_gabo,
                           id_reporte_ventas,
                           )


warnings.filterwarnings('ignore')



In [ ]:
if from_drive:
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    import gspread
    from google.auth import default
    from gspread_dataframe import set_with_dataframe
    import gdown
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    creds, _ = default()
    gc = gspread.authorize(creds)

In [ ]:
atlas_consumo_ids = list_file_ids_for_drive_folder(drive,atlas_consumo_output_folder_id)
id_edas = '13avOAwqLvec7h301gOQdgbA8iVKMKstH_zzggaYYFMw'
id_torre_de_control = '1qitnBwGCg7n-IFPaOBZfdBK2vCplNTqPOjpiJcoZX7k'
folder_id_folios_rod = '1OW4yxE7h8BCcn0mqhCfk05B4_27Ghvfd'
files_rod = list_file_ids_for_drive_folder(drive,folder_id_folios_rod)
bauto = read_from_google_sheets(gc,atlas_consumo_ids['AcConsolidadoBautoLastStatus'],sheetname='Hoja 1') 
edas = read_from_google_sheets(gc,id_edas)
files = list(files_rod.keys())
files
latest_rod_id = files_rod['Solicitudes de crédito 01-08-2025 al 09-12-2025-09_15.xlsx'] ## cambiar este siempre
from_drive_to_local(drive,latest_rod_id,'rod_latest.xlsx')
rod = (pd.read_excel('rod_latest.xlsx')
      .rename(columns={'Name':'intento',
                       'MX_ATN_Id_Simulacion__c':'n_simulacion',
                       'MX_ATN_creditId__c':'folio',
                       'MX_ATN_Account__r.Name':'name',
                       'MX_ATN_Account__r.MX_ATN_CommerceId__c':'id_am',
                       'MX_ATN_Account__r.MX_ATN_PrimaryContact__r.Email':'email',
                       'MX_ATN_Account__r.MX_ATN_PrimaryContact__r.MobilePhone':'phone',
                       'MX_ATN_Status__c':'status'})
      .assign(CreatedDate = lambda x: pd.to_datetime(x.CreatedDate, format="%d/%m/%Y, %H:%M"),
              phone = lambda x: x.phone.astype('Int64').astype(str))
      )
 
torre = read_from_google_sheets(gc,id_torre_de_control, 'Asignación compradores')

latest_rod_id
# rod_dates = [x.split('.xlsx')[0].split('al ')[-1] for x in files_rod.keys()]
# rod_dates = [x for x in rod_dates if  not (x.startswith('Solicitudes')|x.startswith('Rev'))]
# rod_dates = [x[:10] for x in rod_dates]
# rod_dates.sort()
## clean

torre.columns = [x.lstrip(' ').rstrip(' ') for x in torre.columns]
torre_subset_columns = ['ID Lead','Origen AutoMarket','Folio BAuto','ID comprador',
                        'Nombre comprador','Mail Comprador','Teléfono comprador',
                        'Lead Contactado (Visualiza CC)','Estatus de crédito']
rename_dict = {'Folio BAuto':'folio',
               'Nombre comprador':'nombre_comprador_tc',
               'Mail Comprador':'email',
               'Teléfono comprador':'telefono',
               'ID comprador':'id_am',
               'Estatus de crédito':'status_de_credito',
               'Lead Contactado (Visualiza CC)':'visualiza_cc',
               'ID Lead':'id_lead'
               }
torre_mod = (torre[torre_subset_columns]
.rename(columns=rename_dict)
.assign(folio = lambda x: pd.to_numeric(x['folio'],errors='coerce').astype('Int64'),
      id_am = lambda x: pd.to_numeric(x['id_am'],errors='coerce').astype('Int64'),
      telefono = lambda x: np.where(x.telefono=='x',np.nan,x.telefono),
      email = lambda x: np.where(x.email=='x',np.nan,x.email)
      )
)
edas_mod = (edas
            [lambda x: x['Folio Preautorizado'].notna()]
            .assign(fecha_ref= lambda x: pd.to_datetime(x['Marca temporal'].str[0:10].str.strip().str.zfill(10),format='%d/%m/%Y').dt.strftime('%Y-%m-%d'),
                    folio = lambda x: pd.to_numeric(x['Folio Preautorizado'],errors='coerce').astype('Int64')
                    )
            .sort_values(by='fecha_ref',ascending=False)
            .groupby('folio').head(1)
            )
torre_mod.dtypes
bauto_mod = (bauto
 .assign(telefono = lambda x: np.where(x.telefono.astype('Int64').isna(),np.nan,x.telefono.astype('Int64').astype(str)),
         email = lambda x: x.email.str.lower())
 )

torre_mod.dtypes
## transform

bauto.shape[0],bauto.folio.nunique()
pd.set_option('display.max_columns', None)
edas_mod.shape[0],edas_mod.folio.nunique()
torre_mod[lambda x: x.folio.notna()].folio.value_counts()[lambda x: x>1]
torre_mod[lambda x: x.folio==8799612]
(bauto_mod
 .assign(flag_eda = lambda x: np.where(x.folio.isin(edas_mod.folio.unique()),1,0))
 .merge(torre_mod,on='folio',how='left')
#  .merge(torre_mod[['id_am','email']],on='email',how='left')

 )